In [24]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta

# Function to Load data to Mysql

In [25]:
# Function to load data to Mysql
def load_MySQL(table_name,new_dataframe):

    # Replace the following with your own database connection details
    db_username = 'u681559533_AMT'
    db_password = 'MVT2024VTise'
    db_host = '193.203.166.160'
    db_port = '3306'
    db_name = 'u681559533_VT_LF_DB'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Load the DataFrame into MySQL table
    new_dataframe.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=100_000)

    print("DataFrame has been loaded into the database.")

    return None

# Function to create simulation

In [26]:
 # Function to create simulation
# The parameters of the function can be change so it can simulate different values
def create_simulation(date_start, # It is  the date when the simulation start
                      date_finish, # It is the date when the simulation finish
                      mu = 1, # It is the middle value that the distribution is going to take
                      sigma = 1, # It is the variance of the simulation
                      min = 0, # It is the min value that the simulation can take
                      max = 2): # It is the max value that the simulation can take

    # Create date range
    range_dates = pd.date_range(start=date_start, end=date_finish, freq='s')

    # Create DataFrame
    df_dates = pd.DataFrame(range_dates, columns=['timestamp'])

    # Generate the random normal distribution
    random_num = np.random.normal(mu, sigma, size=len(df_dates))

    # Add column to DataFrame
    df_dates['value'] = random_num

    df_dates['value'] = np.where(df_dates['value'] < min, min, df_dates['value'])

    df_dates['value'] = np.where(df_dates['value'] > max, max, df_dates['value'])
    
    return df_dates


# Funcrion to adjust dataframe to XML data structure

In [27]:
# Funcrion to adjust dataframe to XML data structure
def complete_df(df1, df2, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence):
    dft = pd.concat([df1, df2])
    dft['deviceName'] = deviceName
    dft['deviceUuid'] = deviceUuid
    dft['component'] = component
    dft['componentName'] = componentName
    dft['componentId'] = componentId
    dft['measurementType'] = measurementType
    dft['dataItemId'] = dataItemId
    dft['name'] = name
    dft['sequence'] = sequence
    
    dft = dft[['deviceName', 'deviceUuid', 'component', 'componentName', 'componentId', 'measurementType', 'dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value', 'mantenance']]
    return dft

# Read Data

In [28]:
root = ET.parse('StratasysSample.xml')

# Prepare a list to hold extracted data
data = []

# Iterate through each DeviceStream and its ComponentStreams
for device_stream in root.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}DeviceStream'):
    device_name = device_stream.attrib.get('name')
    device_uuid = device_stream.attrib.get('uuid')
    for component_stream in device_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}ComponentStream'):
        component = component_stream.attrib.get('component')
        component_name = component_stream.attrib.get('name')
        component_id = component_stream.attrib.get('componentId')
        # Extract samples
        for samples in component_stream.findall('.//{urn:mtconnect.org:MTConnectStreams:1.5}Samples'):
            for sample in samples:
                # Extract all attributes of the sample
                sample_data = {attr: sample.attrib.get(attr) for attr in sample.attrib}
                sample_data['value'] = sample.text
                sample_data['componentName'] = component_name
                sample_data['componentId'] = component_id
                sample_data['deviceName'] = device_name
                sample_data['deviceUuid'] = device_uuid
                sample_data['component'] = component
                sample_data['measurementType'] = sample.tag.split('}')[1]  # To get 'Amperage', 'Voltage', etc.
                data.append(sample_data)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

df = df[['deviceName','deviceUuid','component','componentName','componentId','measurementType','dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value']].copy()

## Create Tables

In [29]:
# exploration of the table
df_amperage_enclosure = df[(df['measurementType'] == 'Amperage')&(df['component'] == 'Enclosure')]
df_amperage_linear = df[(df['measurementType'] == 'Amperage')&(df['component'] == 'Linear')]
df_voltage = df[df['measurementType'] == 'Voltage']
df_temperature = df[df['measurementType'] == 'Temperature']
df_rotaryvelocity = df[df['measurementType'] == 'RotaryVelocity']
df_powerfactor = df[df['measurementType'] == 'PowerFactor']

# Simulation

In [30]:
# Parameters of simulation
number_of_days = 0
start_date = (datetime.now()- timedelta(days=number_of_days,hours=0,minutes=15)).strftime('%Y-%m-%d %H:%M:%S')
finish_date = (datetime.now()).strftime('%Y-%m-%d %H:%M:%S')

## Temperature

In [31]:
# Parameters of machine
dataItemId = 'ovenHeater1Com'
name = 'OvenHeater1CommandedTemperature'
measurementType = 'Temperature'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2) 
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [32]:
# Parameters of machine
dataItemId = 'ovenHeater2Com'
name = 'OvenHeater2CommandedTemperature'
measurementType = 'Temperature'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(1/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [33]:
# Create temperature dataframe
df_temperature = pd.concat([df_final1, df_final2])

## Speed

In [34]:
# Parameters of machine
dataItemId = 'ovenBlower1Speed'
name = 'OvenBlower1Speed'
measurementType = 'RotaryVelocity'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['maintenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [35]:
# Parameters of machine
dataItemId = 'ovenBlower2Speed'
name = 'OvenBlower2Speed'
measurementType = 'RotaryVelocity'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(1/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [36]:
# create speed dataframe
df_speed = pd.concat([df_final1, df_final2])

## Linear

In [37]:
# Parameters of machine
dataItemId = 'x1Current'
name = 'MotorCurrent'
measurementType = 'Amperage'
component = 'Linear'
componentName = 'X'
componentId = 'x1'
maintenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_target = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['mantenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final1 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [38]:
# Parameters of machine
dataItemId = 'z1Current'
name = 'MotorCurrent'
measurementType = 'Amperage'
component = 'Linear'
componentName = 'Z'
componentId = 'z1'
mantenance_date = (datetime.now()- timedelta(days=number_of_days*(2/3))).strftime('%Y-%m-%d %H:%M:%S')
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688

# Simulation
df_targuet = create_simulation(start_date,finish_date, mu = 2, sigma = 1, min = 0, max = 2)
df_targuet['subType'] = 'TARGET'
df_targuet['mantenance'] = 0

df_actual = create_simulation(start_date,finish_date, mu = 1, sigma = 0.5, min = 0, max = 2)
df_actual['subType'] = 'ACTUAL'
df_actual['maintenance'] = np.where(df_actual['timestamp']==mantenance_date, 1, 0)

df_final2 = complete_df(df_targuet, df_actual, deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence)

In [39]:
# Create linear dataframe
df_linear = pd.concat([df_final1, df_final2])

## Join all tables

In [40]:
# Create final dataframe to load to MySQL
df_final = pd.concat([df_temperature, df_speed, df_linear])

# Load Data

In [41]:
print("Rows to load to MySQL:", len(df_final))
print("Expected time:", (len(df_final)/1_000_000)*1.15, "minutes")

Rows to load to MySQL: 10812
Expected time: 0.0124338 minutes


In [42]:
# Load the data to MySQL
load_MySQL('Stratasys',df_final)

DataFrame has been loaded into the database.
